In [ ]:
import requests
import re

def doPost(uri, pageIdx, pre):
    data = {
        'pageIndex': pageIdx,
        'pageUnit': '10000',
        'pre': pre
    }
    res = requests.post(uri, data=data, verify=False).json()
    return res
    #print(res)

#response의 모든row를 순회하면서 data저장
def appendList(res, h, g):
    last_page = res['a']['b']
    totalRecordCount = res['a']['c']
    print("total page number:" + str(last_page) + ", totalRecordCount:", totalRecordCount)
    print()

    item_list = res['d']['e']

    p = re.compile('[0-9]*')
    tot_word_cnt = 0
    for i in range(len(item_list)):
        h1 = p.findall(str(item_list[i]['f']).strip())
        h2 = ''.join(h1)
        h.append(h2[0:10])
        gd = str(item_list[i]['g']).strip()
        g.append(gd)
        
        w_cnt = len(gd.split(" "))
        tot_word_cnt += w_cnt
    
    avg_word_cnt = tot_word_cnt/len(item_list)
    return last_page, avg_word_cnt

def doSleep(sec):
    sleep_sec = uniform(sec, sec+1)
    print("sleep : " + str(round(sleep_sec, 2)) + "s")
    sleep(sleep_sec)

def createDataFrame(h, g):
    import pandas as pd
    d1 = pd.DataFrame(h)
    d1.columns=['h']
    d2 = pd.DataFrame(g)
    d2.columns=['g']
    d3 = pd.concat([d1, d2],axis=1)

    print(d3.head())
    print(len(d3))
    return d3

def writeCSV(dataFrame, file_name, mode='w'):
    dataFrame.to_csv(file_name, mode=mode, index=False)
    print('export csv success! row:', len(dataFrame))

In [ ]:
from random import uniform
from time import sleep

uri  = ''
h=[]
g=[]
file_name = "1.csv"

#0~9 까지 한자리씩 입력하고 조회
tot_word_cnt = 0
avg_word_cnt = 0
req_cnt = 0
for pre in range(0, 9):
    res = doPost(uri, str(1), pre)
    last_page, avg_word_cnt = appendList(res, h, g)
    print("pre:", pre, " ,pIdx:" + str(1), ' ,avg_word_cnt:', avg_word_cnt)
    tot_word_cnt += avg_word_cnt
    doSleep(5.1)
    
    #2번째 page부터 마지막page까지 쿼리를 날리면서 저장
    for pIdx in range(2, int(last_page) + 1):
        res = doPost(uri, str(pIdx), pre)
        tmp, avg_word_cnt = appendList(res, h, g)
        print("pre:", pre, " ,pIdx:" + str(pIdx), ' ,avg_word_cnt:', avg_word_cnt)
        tot_word_cnt += avg_word_cnt
        req_cnt += 1
        
        if req_cnt!=0 and req_cnt%100==0:
            d3 = createDataFrame(h, g)
            writeCSV(d3, file_name)
            doSleep(5.3)
        doSleep(5.2)
        if pIdx >= 1: break
            
    tot_avg_word_cnt = tot_word_cnt/int(last_page)
    tot_word_cnt = 0
    print('tot_avg_word_cnt:', tot_avg_word_cnt)
    if(pre>=1): break

d3 = createDataFrame(h, g)
writeCSV(d3, file_name)
